In [1]:
import os
import numpy as np

import tensorflow as tf

In [38]:
# create a list to all tensorboard log file
# logs_directory = "../../tmp/tensorboard"
logs_directory = "../tensorboard/osirim_scallableCnn/scallable2"

all_log_dir = os.listdir(logs_directory)

all_log_path = []
for log_dir in all_log_dir:
    log_dir_path = os.path.join(logs_directory, log_dir)
    all_log_path.append(os.path.join(log_dir_path, os.listdir(log_dir_path)[0]))

In [44]:
tags = ["train/acc", "validation/acc"]

def get_log_data(path):
    logs = dict(zip(tags, [[] for _ in range(len(tags))]))

    for e in tf.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag in tags:
                    logs[value.tag].append(value.simple_value)
    
    return logs

# Get all the logs

In [45]:
all_logs = dict(zip(all_log_path, [None for _ in range(len(all_log_path))]))

# for path in all_log_path:
for path in all_log_path:
    all_logs[path] = get_log_data(path)


# Calculate the stats for all the logs

In [46]:
all_logs_stats = dict(zip(all_log_path, [{} for _ in range(len(all_log_path))]))

for path in all_log_path:
    for tag in all_logs[path]:
        try: 
            all_logs_stats[path][tag] = {"max": None, "min": None, "mean": None, "std": None}
            all_logs_stats[path][tag]["max"] = np.asarray(all_logs[path][tag]).max()
            all_logs_stats[path][tag]["min"] = np.asarray(all_logs[path][tag]).min()
            all_logs_stats[path][tag]["mean"] = np.asarray(all_logs[path][tag]).mean()
            all_logs_stats[path][tag]["std"] = np.asarray(all_logs[path][tag]).std()
        except ValueError as e:
            print("Error for tag %s in path %s" % (tag, path))
            print("continue...")
            pass

In [47]:
# search in the stat
def get_of(tag, key, search: str = None) -> list:
    output = []
    
    for path in all_logs_stats.keys():
        if search is not None:
            if search in path:
                output.append(all_logs_stats[path][tag][key])
        else:
            output.append((path, all_logs_stats[path][tag][key]))
    
    return output

In [49]:
target = "max"
response = get_of("validation/acc", target)

# trim path from everythinfg irelevant
# response = [(response[i][0].split("/")[4].split("_")[2], response[i][1]) for i in range(len(response))]

# order by run
# response = sorted(response, key=lambda x: x[0].split("/")[4].split("_")[2])

values = np.array([v[1] for v in response])
path = [p[0] for p in response]

values = np.array([i for i in values if i is not None])
print("Overall statistics --------")
print("mean: ", values.mean())
print("std:  ", values.std())
print("min:  ", values.min())
print("max:  ", values.max())
print("")
print("%s values --------" % target)
for p, v in response:
    print(p, v)

Overall statistics --------
mean:  77.19404983520508
std:   4.480183431459999
min:   71.0
max:   84.32221221923828

max values --------
../tensorboard/osirim_scallableCnn/scallable2/2020-01-14_11:22:47_ScalableCnn1_run6_Cosd-0.05lr_200e_1a_1b_1g_noaugment/events.out.tfevents.1578997367.co2-slurm-ng04.162690.0 84.32221221923828
../tensorboard/osirim_scallableCnn/scallable2/2020-01-14_12:51:15_ScalableCnn1_run10_Cosd-0.05lr_200e_1a_1b_1g_noaugment/events.out.tfevents.1579002675.co2-slurm-ng02.34637.0 79.4779281616211
../tensorboard/osirim_scallableCnn/scallable2/2020-01-14_10:37:59_ScalableCnn1_run3_Cosd-0.05lr_200e_1a_1b_1g_noaugment/events.out.tfevents.1578994679.co2-slurm-ng02.186109.0 75.0
../tensorboard/osirim_scallableCnn/scallable2/2020-01-14_11:22:54_ScalableCnn1_run5_Cosd-0.05lr_200e_1a_1b_1g_noaugment/events.out.tfevents.1578997374.co2-slurm-ng02.4668.0 75.6231918334961
../tensorboard/osirim_scallableCnn/scallable2/2020-01-14_12:07:20_ScalableCnn1_run8_Cosd-0.05lr_200e_1a_1b_1g